## Load Dependencies

In [1]:
# load dependencies
import pandas as pd 
import torch
import torch.nn as nn
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, confusion_matrix

2023-04-10 00:51:33.273624: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Dataset

In [53]:
# load the dataset
url = "iris.data"

# column names to use
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# read the dataset from URL
dataset = pd.read_csv(url, names=names) 

# check first few rows of data
dataset.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Label Encode Output Column

In [54]:
# seperate the independent and dependent features
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values 

# encode output variable
le = LabelEncoder()
y = le.fit_transform(y)

print(le.classes_)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


## Split Data and Standradize Features

In [26]:
# Split dataset into random training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) 

# feature standardization
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

## Convert Data with NumPy Arrays to Tensors

In [38]:
# convert data numpy arrays to tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

## Define Model Architecture

In [39]:
# define model architecture
class NeuralNetworkClassificationModel(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(NeuralNetworkClassificationModel,self).__init__()
        self.input_layer    = nn.Linear(input_dim,128)
        self.hidden_layer1  = nn.Linear(128,64)
        self.output_layer   = nn.Linear(64,output_dim)
        self.relu = nn.ReLU()
    
    
    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

## Define Model Input and Output

In [40]:
# define input and output dimensions
input_dim  = 4 
output_dim = 3
model = NeuralNetworkClassificationModel(input_dim,output_dim)

## Define Model Configuration and Training Procedure

In [42]:
# create our optimizer and loss function object
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

# define training steps
def train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses):
    for epoch in range(num_epochs):
        # clear out the gradients from the last step loss.backward()
        optimizer.zero_grad()
        
        # forward feed
        output_train = model(X_train)

        # calculate the loss
        loss_train = criterion(output_train, y_train)
        

        # backward propagation: calculate gradients
        loss_train.backward()

        # update the weights
        optimizer.step()

        
        output_test = model(X_test)
        loss_test = criterion(output_test,y_test)

        train_losses[epoch] = loss_train.item()
        test_losses[epoch] = loss_test.item()

        if (epoch + 1) % 50 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss_train.item():.4f}, Test Loss: {loss_test.item():.4f}")

## Train Pytorch Based Model

In [45]:
num_epochs = 1000
train_losses = np.zeros(num_epochs)
test_losses  = np.zeros(num_epochs)

In [46]:
train_network(model,optimizer,criterion,X_train,y_train,X_test,y_test,num_epochs,train_losses,test_losses)

Epoch 50/1000, Train Loss: 0.0228, Test Loss: 0.1840
Epoch 100/1000, Train Loss: 0.0018, Test Loss: 0.2661
Epoch 150/1000, Train Loss: 0.0001, Test Loss: 0.2478
Epoch 200/1000, Train Loss: 0.0000, Test Loss: 0.2535
Epoch 250/1000, Train Loss: 0.0000, Test Loss: 0.2607
Epoch 300/1000, Train Loss: 0.0000, Test Loss: 0.2629
Epoch 350/1000, Train Loss: 0.0000, Test Loss: 0.2645
Epoch 400/1000, Train Loss: 0.0000, Test Loss: 0.2661
Epoch 450/1000, Train Loss: 0.0000, Test Loss: 0.2669
Epoch 500/1000, Train Loss: 0.0000, Test Loss: 0.2689
Epoch 550/1000, Train Loss: 0.0000, Test Loss: 0.2703
Epoch 600/1000, Train Loss: 0.0000, Test Loss: 0.2702
Epoch 650/1000, Train Loss: 0.0000, Test Loss: 0.2709
Epoch 700/1000, Train Loss: 0.0000, Test Loss: 0.2731
Epoch 750/1000, Train Loss: 0.0000, Test Loss: 0.2729
Epoch 800/1000, Train Loss: 0.0000, Test Loss: 0.2718
Epoch 850/1000, Train Loss: 0.0000, Test Loss: 0.2731
Epoch 900/1000, Train Loss: 0.0000, Test Loss: 0.2740
Epoch 950/1000, Train Loss: 0

## Save Model Weights and Architecture

In [50]:
# save model 
torch.save(model, 'model_pytorch.pt')

## Load Model Weights and Architecture

In [51]:
# load model
model = torch.load('model_pytorch.pt')
# check model summary
model.eval()

NeuralNetworkClassificationModel(
  (input_layer): Linear(in_features=4, out_features=128, bias=True)
  (hidden_layer1): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=3, bias=True)
  (relu): ReLU()
)